In [ ]:
import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from requests_html import HTML

________________________________________

In [ ]:
url = "https://soundcloud.com/discover"

In [ ]:
browser = webdriver.Chrome()
browser.get(url)

In [ ]:
soup = BeautifulSoup(browser.page_source)

In [ ]:
tracks = []
playlists = []

In [ ]:
# nên check lại bởi vì vài lần chưa scroll hết :<
def scroll_down_till_end():
    # Loop 2 lần cho chắc ăn (có thể cho 3 lần)
    for i in range(2):
        lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        lastCount = 0
        match = False
        
        while(match == False):
            time.sleep(3)
            lenOfPage = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            if lastCount == lenOfPage: 
                match = True
            else: 
                lastCount = lenOfPage

In [ ]:
# nên check lại bởi vì vài lần chưa scroll hết :<
def scroll_up():
    # Loop 2 lần cho chắc ăn (có thể cho 3 lần)
    for i in range(2):
        lenOfPage = browser.execute_script("window.scrollTo(document.body.scrollHeight, 0);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
        lastCount = 0
        match = False
        
        while(match == False):
            time.sleep(3)
            lenOfPage = browser.execute_script("window.scrollTo(document.body.scrollHeight,0);var lenOfPage=document.body.scrollHeight;return lenOfPage;")
            if lastCount == lenOfPage: 
                match = True
            else: 
                lastCount = lenOfPage

In [ ]:
def list_to_tsv_csv(listItem, outputFile):
    df = pd.DataFrame(listItem)
    df.to_csv(outputFile, mode='a', sep='\t')

In [ ]:
def click_next_button():
    button = browser.find_element(By.XPATH,"//button[@class='tileGallery__sliderButton tileGallery__slideForwardButton']")
    button.click()

In [ ]:
# Đồng ý cookies
cookies = browser.find_element(By.XPATH,"//button[@id='onetrust-accept-btn-handler']")
cookies.click()
    
# Chọn một tab genre (demo)
time.sleep(2)
genre = browser.find_element(By.XPATH,"//a[@class='playableTile__heading playableTile__mainHeading sc-truncate sc-type-light sc-text-secondary sc-font-light sc-link-dark sc-link-primary sc-text-h4']")
genre.click()

soup = BeautifulSoup(browser.page_source)

time.sleep(2)
profile = browser.find_element(By.XPATH,"//a[@class='trackItem__username sc-link-light sc-link-secondary']")
profile.click()

**Lấy tất cả tracks của một user**

In [ ]:
def get_all_tracks():
    scroll_down_till_end()
    time.sleep(3)
    soup = BeautifulSoup(browser.page_source)
    
    time.sleep(3)
    songs = soup.find_all("li", "soundList__item")
    for track in songs:
        title = track.find("a", "soundTitle__title").get_text().strip()
        # kiểm tra có bị trùng
        if title not in tracks:
            tracks.append(title)

**Lấy tất cả playlists của một user**

In [ ]:
def get_all_playlists():
    # Xuất phát từ cuối trang tracks nên phải scroll up lên đầu trang
    scroll_up()
    time.sleep(3)
    
    soup = BeautifulSoup(browser.page_source)

    time.sleep(2)
    tab_playlists = browser.find_element(By.XPATH,'//a[contains(@href,"/winnodamidside/sets")]')
    tab_playlists.click()
    
    time.sleep(3)
    scroll_down_till_end()
    soup = BeautifulSoup(browser.page_source)
    
    time.sleep(3)
    playlists = soup.find_all("li", "soundList__item")
    for playlist in playlists:
        title = playlist.find("a", "soundTitle__title").get_text().strip()
        if title is None:
            break
        # kiểm tra có bị trùng
        if title not in playlists:
            print(title)
            playlists.append(title)

In [ ]:
get_all_tracks()

In [ ]:
scroll_up()

In [ ]:
get_all_playlists()

**Lưu data vào file tsv**

In [ ]:
outputFile = "Api_data/track.csv"
list_to_tsv_csv(tracks, outputFile)

In [ ]:
outputFile = "Api_data/playlist.csv"
list_to_tsv_csv(playlists, outputFile)

In [ ]:
soup = BeautifulSoup(browser.page_source)
genres = soup.find_all("li", "mixedModularHome__item")

for i in range(len(genres)):
    lists = genres[i].find_all("div", "tileGallery__sliderPanelSlide")
    j = 4
    while True:
        if j > len(lists):
            break
        browser.find_elements(By.XPATH,"//button[contains(.,'forward')]")[i].click()
        j += 4

In [ ]:
soup = BeautifulSoup(browser.page_source)  
genres = soup.find_all("li", "mixedModularHome__item")
links = []

for i in range(len(genres)):
    lists = genres[i].find_all("div", "tileGallery__sliderPanelSlide")
    for pl in lists:
        title = pl.find("a", "playableTile__heading").get_text().strip()
        playlists.append(title)
        link = pl.find("a", "playableTile__artworkLink")["href"]
        links.append("https://soundcloud.com" + link)

In [ ]:
playlists

In [ ]:
links

In [ ]:
for link in links:
    browser.get(link)
    time.sleep(5)
    browser.back()